In [ ]:
!pip install lightfm

In [2]:
import numpy as np

from lightfm.datasets import fetch_movielens

data = fetch_movielens(min_rating=5.0)


c:\Users\biaspaltsau_aa\Anaconda3\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


Это загружает набор данных и автоматически предварительно обрабатывает его в разреженные матрицы, подходящие для дальнейшего расчета. В частности, он подготавливает разреженные матрицы элементов пользователя, содержащие положительные записи, когда пользователь взаимодействовал с продуктом, и нули в противном случае.

У нас есть две такие матрицы, обучающая и тестовая выборка. Оба имеют около 1000 пользователей и 1700 элементов. Мы будем обучать модель на матрице поездов, но тестировать ее на тестовой матрице.

In [6]:
data['train'].data.shape

(19048,)

In [8]:
data['test'].data.shape

(2153,)

Мы собираемся использовать модель WARP (взвешенный приблизительный ранг попарно). WARP — это модель неявной обратной связи: все взаимодействия в обучающей матрице трактуются как положительные сигналы, а продукты, с которыми пользователи не взаимодействовали, им неявно не нравятся. Цель модели состоит в том, чтобы высоко оценить эти неявные позитивы и присвоить низкие баллы неявно негативным.

Обучение модели осуществляется с помощью SGD (стохастический градиентный спуск). Это означает, что для каждого прохода данных --- эпохи --- модель учится подгонять данные все более и более точно. В этом примере мы запустим его на 30 эпох. Мы также можем запустить его на нескольких ядрах, поэтому мы установим его равным 8

In [17]:
from lightfm import LightFM

In [18]:
model = LightFM(loss='warp')
model.fit(data['train'], epochs=30, num_threads=8)

Выполнено! Теперь мы должны оценить модель, чтобы увидеть, насколько хорошо она работает. Нас больше всего интересует, насколько хорош рейтинг, полученный моделью. Precision@k — одна из подходящих метрик, выражающая процент k лучших элементов в рейтинге, с которыми пользователь фактически взаимодействовал. lightfm реализует ряд показателей в модуле оценки.

In [20]:
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score

In [15]:
print("Train precision: %.2f" % precision_at_k(model, data['train'], k=5).mean())
print("Test precision: %.2f" % precision_at_k(model, data['test'], k=5).mean())

Train precision: 0.40
Test precision: 0.05


In [22]:
print("Train precision: %.2f" % auc_score(model, data['train']).mean())
print("Test precision: %.2f" % auc_score(model, data['test']).mean())

Train precision: 0.97
Test precision: 0.92


Неудивительно, что модель подходит для набора поездов лучше, чем тестовый набор.

В качестве альтернативного способа оценки модели мы можем выбрать пару пользователей и получить их рекомендации. Чтобы делать прогнозы для данного пользователя, мы передаем идентификатор этого пользователя и идентификаторы всех продуктов, для которых мы хотим прогнозировать, в метод прогнозирования.

In [27]:
def sample_recommendation(model, data, user_ids):
    

    n_users, n_items = data['train'].shape

    for user_id in user_ids:
        known_positives = data['item_labels'][data['train'].tocsr()[user_id].indices]
        
        scores = model.predict(user_id, np.arange(n_items))
        top_items = data['item_labels'][np.argsort(-scores)]
        
        print("User %s" % user_id)
        print("     Known positives:")
        
        for x in known_positives[:3]:
            print("        %s" % x)

        print("     Recommended:")
        
        for x in top_items[:3]:
            print("        %s" % x)
        
sample_recommendation(model, data, [3, 25, 450]) 

User 3
     Known positives:
        Contact (1997)
        Air Force One (1997)
        In & Out (1997)
     Recommended:
        Air Force One (1997)
        Scream (1996)
        G.I. Jane (1997)
User 25
     Known positives:
        Fargo (1996)
        Godfather, The (1972)
        L.A. Confidential (1997)
     Recommended:
        Fargo (1996)
        L.A. Confidential (1997)
        Titanic (1997)
User 450
     Known positives:
        Event Horizon (1997)
        Scream (1996)
        Conspiracy Theory (1997)
     Recommended:
        Game, The (1997)
        Contact (1997)
        Air Force One (1997)
